In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Fonctions

In [ ]:
def create_benchmark_model():

    inputs = layers.Input(shape=(100, 19))

    gru_fwd = layers.GRU(64, return_sequences=False)(inputs)
    gru_bwd = layers.GRU(64, return_sequences=False, go_backwards=True)(inputs)

    concatenated = layers.concatenate([gru_fwd, gru_bwd])
    dense_1 = layers.Dense(64, activation='selu')(concatenated)
    dense_2 = layers.Dense(128, activation='selu')(dense_1)
    dense_3 = layers.Dense(128, activation='selu')(dense_2)
    dense_4 = layers.Dense(64, activation='selu')(dense_3)
    output = layers.Dense(24, activation='softmax')(dense_4)

    model = tf.keras.Model(inputs=inputs, outputs=output)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-3),
                loss='sparse_categorical_crossentropy')
    
    return model

# Création de la base de donnée

In [ ]:

pd.set_option('display.max_columns', None)

df=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/X_train_N1UvY30.csv")  
df_result=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/y_train_or6m3Ta.csv")   

df['bid_ask_spread']=df['ask']-df['bid']
df['trade']=df['trade'].astype(int)

value_categor=['venue','action','side']

for k in value_categor:

    df_encoded = pd.get_dummies(df[k], prefix=k).astype(int)
    df = df.merge( df_encoded , left_index = True , right_index = True)
    df.drop( columns=[k] , inplace = True )

df.drop(columns=['obs_id','order_id'],inplace=True)

value_no_center=['price' , 'bid' , 'ask' , 'bid_size', 'ask_size', 'bid_ask_spread' ]

for k in value_no_center:
    
    df[k] = df[k] / max( df[k] )

In [ ]:
X = df.values.reshape(160800, 100, 19)
y = df_result['eqt_code_cat'].values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Mise en place d'un modèle

In [ ]:
model = create_benchmark_model()
history = model.fit(x_train, y_train, batch_size = 1000, epochs = 10)

In [ ]:
prediction_prob = model.predict(x_test)
y_pred = []
for k in prediction_prob :
    y_pred += [ k.argmax() ]

In [ ]:
accuracy_score(y_pred,y_test)